<a href="https://colab.research.google.com/github/ramnarayan-code/llm_reflection/blob/main/langgraph_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
%%capture --no-stderr
%pip install -U langchain langgraph langchain_openai langchain_experimental langsmith pandas

In [4]:
import os
os.environ['OPENAI_API_KEY'] = ''
print(os.getenv('OPENAI_API_KEY'))
# https://medium.com/@anuragmishra_27746/future-of-coding-multi-agent-llm-framework-using-langgraph-092da9493663

sk-proj-bbQrZmLH0v98r2IrpHlAT3BlbkFJT8ZsruIC0LvEBGjy7yft


In [18]:
import os
from langchain.tools import DuckDuckGoSearchRun
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt import create_agent_executor
from langchain_core.pydantic_v1 import BaseModel
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Annotated, Any, Dict, Optional, Sequence, TypedDict, List, Tuple
import operator


In [27]:
llm=ChatOpenAI()

In [36]:
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate

class ProductOwner(BaseModel):

    user_stories: str = Field(
        description="Detailed user stories"
    )
product_owner_prompt = ChatPromptTemplate.from_template(
    '''
    **Role**: You are a product owner. You need to create user stories
**Task**: As a product owner, you are required to create 5 user stories based on the use case

**Instructions**:
1. **Understand and Clarify**: Make sure you understand the use case.
2. **Define user story and assign priority**: define user story in the below format following the agile standards and assign the priority level out of the values: high, medium, low.
*FORMAT*
User story:
Priority:

*USE CASE*
{use_case}

'''
)
product_owner_agent = create_structured_output_runnable(
    ProductOwner, llm, product_owner_prompt
)


In [37]:
user_stories = product_owner_agent.invoke({'use_case':'Design a Video streaming platform'})


In [38]:
print(user_stories.user_stories)

User story 1: As a user, I want to be able to create a profile on the video streaming platform.
Priority: High

User story 2: As a user, I want to be able to search for and discover new videos on the platform.
Priority: High

User story 3: As a user, I want to be able to create playlists and organize my favorite videos.
Priority: Medium

User story 4: As a user, I want to receive personalized video recommendations based on my viewing history.
Priority: Medium

User story 5: As a user, I want to be able to like, comment, and share videos with other users.
Priority: High


In [39]:
class Developer(BaseModel):
    """Plan to follow in future"""

    user_stories_feasibilities: str = Field(
        description="Detailed optmized error-free Python code on the provided requirements"
    )


from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate

developer_prompt = ChatPromptTemplate.from_template(
    '''
    **Role**: You are a developer. You need to assess the user stories
**Task**: As a developer, you need to assess the user stories and provide the technical feasibility report

**Instructions**:
1. **Understand and Clarify**: Make sure you understand the use stories and its respective priority.
2. **Define solution and Estimated Time of Arrival**: propose a high level technical solution and estimated time of arrival(ETA) following the below output format.
*FORMAT*
High level Solution:
ETA:

*USE STORIES*
{user_stories}'''
)
developer_agent = create_structured_output_runnable(
    Developer, llm, developer_prompt
)


In [40]:
developer_report = developer_agent.invoke({'user_stories':user_stories.user_stories})


In [41]:
print(developer_report.user_stories_feasibilities)

High level Solution:
ETA:

User story 1: As a user, I want to be able to create a profile on the video streaming platform.
Priority: High

User story 2: As a user, I want to be able to search for and discover new videos on the platform.
Priority: High

User story 3: As a user, I want to be able to create playlists and organize my favorite videos.
Priority: Medium

User story 4: As a user, I want to receive personalized video recommendations based on my viewing history.
Priority: Medium

User story 5: As a user, I want to be able to like, comment, and share videos with other users.
Priority: High


In [ ]:
class AgentState(TypedDict):
    user_stories: str
    user_stories_feasibilities: str

In [ ]:
def product_owner(state):
    print(f'Entering PO')
    use_case = state['use_case']
    po_outcome = product_owner_agent.invoke({'use_case':use_case})
    return {'user_stories':po_outcome.user_stories}

def developer(state):
    print(f'Entering Dev')
    user_stories = state['user_stories']
    developer_outcome = developer_agent.invoke({'user_stories':user_stories})
    return {'user_stories_feasibilities':developer_outcome.user_stories_feasibilities}

In [ ]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentState)

# Define the nodes
workflow.add_node("product_owner", product_owner)
workflow.add_node("developer", developer)

# Build graph
workflow.set_entry_point("product_owner")
workflow.add_edge("product_owner", "developer")
workflow.add_edge("developer", END)

app = workflow.compile()


In [ ]:

for s in graph.stream(
    {"messages": [HumanMessage(content="Write a brief research report on pikas.")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")